## __Check first before starting__

In [1]:
import os

Working_directory = os.path.normpath("/mnt/mydisk/Continual_Learning_JL/Continual_Learning/")
os.chdir(Working_directory)
print(f"Working directory: {os.getcwd()}")

Working directory: /mnt/mydisk/Continual_Learning_JL/Continual_Learning


## __All imports__

In [2]:
# Operating system and file management
import os
import shutil
import contextlib
import traceback
import gc
import copy
from collections import defaultdict
import subprocess
import time
import re, pickle
import scipy.io
from scipy.io import loadmat
from glob import glob
from math import ceil

# Jupyter notebook widgets and display
import ipywidgets as widgets
from IPython.display import display

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Plotting and visualization
import matplotlib.pyplot as plt
from mpl_interactions import zoom_factory, panhandler

# Machine learning and preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
from ta import trend, momentum, volatility, volume

# Mathematical and scientific computing
import math
from scipy.ndimage import gaussian_filter1d

# Type hinting
from typing import Callable, Tuple

# Deep learning with PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision.models import resnet18
from sklearn.utils.class_weight import compute_class_weight

## __📁 Path Settings and Constants__
This cell defines essential paths and constants for the CPSC2018 ECG dataset processing:
- `BASE_DIR`: Root directory of the project.
- `save_dir`: Path to the preprocessed `.npy` files (one for each continual learning period).
- `ECG_PATH`: Directory containing original `.mat` and `.hea` files.
- `MAX_LEN`: Length of each ECG sample, fixed to 5000 time steps (i.e., 10 seconds at 500Hz).

In [3]:
BASE_DIR = "/mnt/mydisk/Continual_Learning_JL/Continual_Learning/Class_Incremental_CL/CPSC_CIL"
save_dir = os.path.join(BASE_DIR, "processed")
ECG_PATH = os.path.join(BASE_DIR, "datas")
MAX_LEN = 5000

## __🏷️ Label Mapping and Period Configuration__

This section defines:
- `snomed_map`: Mapping from SNOMED CT codes to readable class names for 9 major ECG conditions.
- `period_label_map`: Incremental learning task structure across four periods.  
  Class `1` is reserved for "OTHER" abnormalities until Period 4 when all 9 classes are explicitly categorized.
- `print_class_distribution()`: Helper function to show class-wise data distribution.


In [4]:
# SNOMED CT to readable names
snomed_map = {
    "426783006": "NSR",    # 正常竇性心律
    "270492004": "I-AVB",  # 一度房室傳導阻滯
    "164889003": "AF",     # 心房纖維顫動
    "164909002": "LBBB",   # 左束支傳導阻滯
    "59118001":  "RBBB",   # 右束支傳導阻滯
    "284470004": "PAC",    # 心房早期搏動
    "164884008": "PVC",    # 室性早期搏動
    "429622005": "STD",    # ST 段壓低
    "164931005": "STE"     # ST 段抬高
}

# Period class mapping (固定 class 1 是「其他異常」直到 P4 移除)
period_label_map = {
    1: {"NSR": 0, "OTHER": 1},
    2: {"NSR": 0, "I-AVB": 2, "AF": 3, "OTHER": 1},
    3: {"NSR": 0, "I-AVB": 2, "AF": 3, "LBBB": 4, "RBBB": 5, "OTHER": 1},
    4: {"NSR": 0, "I-AVB": 2, "AF": 3, "LBBB": 4, "RBBB": 5, "PAC": 6, "PVC": 7, "STD": 8, "STE": 9}
}

def print_class_distribution(y, label_map):
    y = np.array(y).flatten()
    total = len(y)
    all_labels = sorted(label_map.values())
    print("\n📊 Class Distribution")
    for lbl in all_labels:
        count = np.sum(y == lbl)
        label = [k for k, v in label_map.items() if v == lbl]
        name = label[0] if label else str(lbl)
        print(f"  ├─ Label {lbl:<2} ({name:<10}) → {count:>5} samples ({(count/total)*100:5.2f}%)")

def ensure_folder(folder_path: str) -> None:
    """Ensure the given folder exists, create it if not."""
    os.makedirs(folder_path, exist_ok=True)


## 📦 EX. Load Example (Period 4) Data and View Format

This example demonstrates how to load preprocessed `.npy` data for **Period 4**, and inspect the dataset shapes and label distribution.  
Use this format as a reference when loading data in other methods (e.g., EWC, PNN, DynEx-CLoRA).

Each ECG sample:
- Has shape `(5000, 12)` → represents 10 seconds (at 500Hz) across 12-lead channels.
- Corresponding label is an integer ID (e.g., 0–9) defined by `period_label_map[4]`.

In [6]:
# 範例:載入 period 4
save_dir = os.path.join(BASE_DIR, "processed")
X_train = np.load(os.path.join(save_dir, "X_train_p4.npy"))
y_train = np.load(os.path.join(save_dir, "y_train_p4.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p4.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p4.npy"))

print("✅ Loaded")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print_class_distribution(y_train, period_label_map[4])
print_class_distribution(y_test, period_label_map[4])

del X_train, y_train, X_test, y_test


✅ Loaded
X_train shape: (5493, 5000, 12)
y_train shape: (5493,)
X_test shape: (1374, 5000, 12)
y_test shape: (1374,)

📊 Class Distribution
  ├─ Label 0  (NSR       ) →   734 samples (13.36%)
  ├─ Label 2  (I-AVB     ) →   577 samples (10.50%)
  ├─ Label 3  (AF        ) →   976 samples (17.77%)
  ├─ Label 4  (LBBB      ) →   158 samples ( 2.88%)
  ├─ Label 5  (RBBB      ) →  1337 samples (24.34%)
  ├─ Label 6  (PAC       ) →   434 samples ( 7.90%)
  ├─ Label 7  (PVC       ) →   501 samples ( 9.12%)
  ├─ Label 8  (STD       ) →   628 samples (11.43%)
  ├─ Label 9  (STE       ) →   148 samples ( 2.69%)

📊 Class Distribution
  ├─ Label 0  (NSR       ) →   184 samples (13.39%)
  ├─ Label 2  (I-AVB     ) →   144 samples (10.48%)
  ├─ Label 3  (AF        ) →   244 samples (17.76%)
  ├─ Label 4  (LBBB      ) →    40 samples ( 2.91%)
  ├─ Label 5  (RBBB      ) →   335 samples (24.38%)
  ├─ Label 6  (PAC       ) →   108 samples ( 7.86%)
  ├─ Label 7  (PVC       ) →   125 samples ( 9.10%)
  ├─ La

## __Check GPU, CUDA, Pytorch__

In [5]:
!nvidia-smi

Mon May  5 17:05:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:2A:00.0 Off |                  Off |
| 30%   49C    P2             80W /  300W |   16904MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### CUDA Details

In [6]:
def check_gpu_config():
    """
    Check GPU availability and display detailed configuration information.
    """
    # Check if GPU is available
    gpu_available = torch.cuda.is_available()
    
    # Print header
    print("=" * 50)
    print("GPU Configuration Check".center(50))
    print("=" * 50)
    
    # Basic GPU availability
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'GPU Available':<25}: {'Yes' if gpu_available else 'No'}")
    
    # If GPU is available, print detailed info
    if gpu_available:
        print("-" * 50)
        print("GPU Details".center(50))
        print("-" * 50)
        
        # Device info
        print(f"{'Device Name':<25}: {torch.cuda.get_device_name(0)}")
        print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")
        print(f"{'Current Device Index':<25}: {torch.cuda.current_device()}")
        
        # Compute capability and CUDA cores
        props = torch.cuda.get_device_properties(0)
        print(f"{'Compute Capability':<25}: {props.major}.{props.minor}")
        print(f"{'Total CUDA Cores':<25}: {props.multi_processor_count * 128}")  # Approx. 128 cores per SM
        
        # Memory info
        total_memory = props.total_memory / (1024 ** 3)  # Convert to GB
        memory_allocated = torch.cuda.memory_allocated(0) / (1024 ** 3)
        memory_reserved = torch.cuda.memory_reserved(0) / (1024 ** 3)
        print(f"{'Total Memory (GB)':<25}: {total_memory:.2f}")
        print(f"{'Allocated Memory (GB)':<25}: {memory_allocated:.2f}")
        print(f"{'Reserved Memory (GB)':<25}: {memory_reserved:.2f}")
    else:
        print("-" * 50)
        print("No GPU detected. Running on CPU.".center(50))
        print("-" * 50)
    
    print("=" * 50)

if __name__ == "__main__":
    check_gpu_config()

             GPU Configuration Check              
PyTorch Version          : 2.5.1
GPU Available            : Yes
--------------------------------------------------
                   GPU Details                    
--------------------------------------------------
Device Name              : NVIDIA RTX A6000
Number of GPUs           : 3
Current Device Index     : 0
Compute Capability       : 8.6
Total CUDA Cores         : 10752
Total Memory (GB)        : 47.41
Allocated Memory (GB)    : 0.00
Reserved Memory (GB)     : 0.00


### PyTorch Details

In [7]:
def print_torch_config():
    """Print PyTorch and CUDA configuration in a formatted manner."""
    print("=" * 50)
    print("PyTorch Configuration".center(50))
    print("=" * 50)
    
    # Basic PyTorch and CUDA info
    print(f"{'PyTorch Version':<25}: {torch.__version__}")
    print(f"{'CUDA Compiled Version':<25}: {torch.version.cuda}")
    print(f"{'CUDA Available':<25}: {'Yes' if torch.cuda.is_available() else 'No'}")
    print(f"{'Number of GPUs':<25}: {torch.cuda.device_count()}")

    # GPU details if available
    if torch.cuda.is_available():
        print(f"{'GPU Name':<25}: {torch.cuda.get_device_name(0)}")

    print("-" * 50)
    
    # Seed setting
    torch.manual_seed(42)
    print(f"{'Random Seed':<25}: 42 (Seeding successful!)")
    
    print("=" * 50)

if __name__ == "__main__":
    print_torch_config()

              PyTorch Configuration               
PyTorch Version          : 2.5.1
CUDA Compiled Version    : 12.1
CUDA Available           : Yes
Number of GPUs           : 3
GPU Name                 : NVIDIA RTX A6000
--------------------------------------------------
Random Seed              : 42 (Seeding successful!)


## __⚙️ GPU Selection — Auto-select the least loaded GPU__
This code automatically scans available GPUs and selects the one with the lowest current memory usage.


In [8]:
def auto_select_cuda_device(verbose=True):
    """
    Automatically selects the CUDA GPU with the least memory usage.
    Falls back to CPU if no GPU is available.
    """
    if not torch.cuda.is_available():
        print("🚫 No CUDA GPU available. Using CPU.")
        return torch.device("cpu")

    try:
        # Run nvidia-smi to get memory usage of each GPU
        smi_output = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'],
            encoding='utf-8'
        )
        memory_used = [int(x) for x in smi_output.strip().split('\n')]
        best_gpu = int(np.argmin(memory_used))

        if verbose:
            print("🎯 Automatically selected GPU:")
            print(f"    - CUDA Device ID : {best_gpu}")
            print(f"    - Memory Used    : {memory_used[best_gpu]} MiB")
            print(f"    - Device Name    : {torch.cuda.get_device_name(best_gpu)}")
        return torch.device(f"cuda:{best_gpu}")
    except Exception as e:
        print(f"⚠️ Failed to auto-detect GPU. Falling back to cuda:0. ({e})")
        return torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Execute and assign
device = auto_select_cuda_device()

🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 14709 MiB
    - Device Name    : NVIDIA RTX A6000


## __Model Selection__

### ResNet 18 - 1D

In [9]:
class ResNet18_1D(nn.Module):
    def __init__(self, input_channels: int, output_size: int):
        super(ResNet18_1D, self).__init__()
        base_model = resnet18(pretrained=False)

        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = base_model.layer1
        self.layer2 = base_model.layer2
        self.layer3 = base_model.layer3
        self.layer4 = base_model.layer4

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(512, output_size)

        self._convert_layers_to_1d()

    def _convert_layers_to_1d(self):
        for name in ['layer1', 'layer2', 'layer3', 'layer4']:
            layer = getattr(self, name)
            for block in layer:
                block.conv1 = nn.Conv1d(block.conv1.in_channels, block.conv1.out_channels,
                                        kernel_size=3, stride=block.conv1.stride[0],
                                        padding=1, bias=False)
                block.bn1 = nn.BatchNorm1d(block.bn1.num_features)
                block.conv2 = nn.Conv1d(block.conv2.in_channels, block.conv2.out_channels,
                                        kernel_size=3, stride=1, padding=1, bias=False)
                block.bn2 = nn.BatchNorm1d(block.bn2.num_features)
                if block.downsample is not None:
                    conv = nn.Conv1d(block.downsample[0].in_channels,
                                     block.downsample[0].out_channels,
                                     kernel_size=1, stride=block.downsample[0].stride[0], bias=False)
                    bn = nn.BatchNorm1d(block.downsample[1].num_features)
                    block.downsample = nn.Sequential(conv, bn)

    def forward(self, x):  # x: (B, T, D)
        x = x.permute(0, 2, 1)  # → (B, D, T)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.global_pool(x).squeeze(-1)  # → (B, 512)
        x = self.classifier(x)               # → (B, num_classes)
        return x


### ResNet 18 - 1D (ResNet18_1D_big_ker)

In [38]:
class ResNet18_1D_big_ker(nn.Module):
    def __init__(self, input_channels: int, output_size: int):
        super(ResNet18_1D_big_ker, self).__init__()
        base_model = resnet18(pretrained=False)

        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=15, stride=2, padding=7, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = base_model.layer1
        self.layer2 = base_model.layer2
        self.layer3 = base_model.layer3
        self.layer4 = base_model.layer4

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(512, output_size)

        self._convert_layers_to_1d()

    def _convert_layers_to_1d(self):
        for name in ['layer1', 'layer2', 'layer3', 'layer4']:
            layer = getattr(self, name)
            for block in layer:
                block.conv1 = nn.Conv1d(block.conv1.in_channels, block.conv1.out_channels,
                                        kernel_size=3, stride=block.conv1.stride[0],
                                        padding=1, bias=False)
                block.bn1 = nn.BatchNorm1d(block.bn1.num_features)
                block.conv2 = nn.Conv1d(block.conv2.in_channels, block.conv2.out_channels,
                                        kernel_size=3, stride=1, padding=1, bias=False)
                block.bn2 = nn.BatchNorm1d(block.bn2.num_features)
                if block.downsample is not None:
                    conv = nn.Conv1d(block.downsample[0].in_channels,
                                     block.downsample[0].out_channels,
                                     kernel_size=1, stride=block.downsample[0].stride[0], bias=False)
                    bn = nn.BatchNorm1d(block.downsample[1].num_features)
                    block.downsample = nn.Sequential(conv, bn)

    def forward(self, x):  # x: (B, T, D)
        x = x.permute(0, 2, 1)  # → (B, D, T)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.global_pool(x).squeeze(-1)  # → (B, 512)
        x = self.classifier(x)               # → (B, num_classes)
        return x


### ResNet 18 - 1D (ResNet18_1D_big_inplane)

In [41]:
class BasicBlock1d(nn.Module):
    expansion = 1
    
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock1d, self).__init__()
        self.conv1 = nn.Conv1d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out

class ResNet18_1D_big_inplane(nn.Module):
    def __init__(self, input_channels=12, output_size=9, inplanes=64):
        super(ResNet18_1D_big_inplane, self).__init__()
        self.inplanes = inplanes
        
        # Initial conv layer
        self.conv1 = nn.Conv1d(input_channels, self.inplanes, kernel_size=15, stride=2, padding=7, bias=False)
        self.bn1 = nn.BatchNorm1d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        # Residual layers
        self.layer1 = self._make_layer(BasicBlock1d, 64, 2)
        self.layer2 = self._make_layer(BasicBlock1d, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock1d, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock1d, 512, 2, stride=2)
        
        # Adaptive pooling (both avg and max)
        self.adaptiveavgpool = nn.AdaptiveAvgPool1d(1)
        self.adaptivemaxpool = nn.AdaptiveMaxPool1d(1)
        
        # Fully connected layer with dropout
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(512 * BasicBlock1d.expansion * 2, output_size)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )
        
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))
            
        return nn.Sequential(*layers)
    
    def forward(self, x):
        # Expect input shape: (batch_size, time_steps, channels)
        x = x.permute(0, 2, 1)  # → (batch_size, channels, time_steps)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        # Apply both avg and max pooling
        x1 = self.adaptiveavgpool(x)
        x2 = self.adaptivemaxpool(x)
        
        # Concatenate pooling results
        x = torch.cat((x1, x2), dim=1)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Apply dropout
        x = self.dropout(x)
        
        # Final classification
        x = self.fc(x)
        
        return x

def ResNet18_1D_big_inplane_def(input_channels=12, num_classes=9):
    """
    Create a ResNet18-like 1D model with both adaptive average and max pooling
    """
    return ResNet18_1D_big_inplane(input_channels=input_channels, output_size=num_classes)

### ResNet 18 - 1D SE

In [10]:
class SEBlock1D(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock1D, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class ResNet18_1D_SE(nn.Module):
    def __init__(self, input_channels: int, output_size: int):
        super(ResNet18_1D_SE, self).__init__()
        base_model = resnet18(pretrained=False)

        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = base_model.layer1
        self.layer2 = base_model.layer2
        self.layer3 = base_model.layer3
        self.layer4 = base_model.layer4

        self.se_blocks = nn.ModuleList([
            SEBlock1D(64), SEBlock1D(128), SEBlock1D(256), SEBlock1D(512)
        ])

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(512, output_size)

        self._convert_layers_to_1d()

    def _convert_layers_to_1d(self):
        for name in ['layer1', 'layer2', 'layer3', 'layer4']:
            layer = getattr(self, name)
            for block in layer:
                block.conv1 = nn.Conv1d(block.conv1.in_channels, block.conv1.out_channels,
                                        kernel_size=3, stride=block.conv1.stride[0],
                                        padding=1, bias=False)
                block.bn1 = nn.BatchNorm1d(block.bn1.num_features)
                block.conv2 = nn.Conv1d(block.conv2.in_channels, block.conv2.out_channels,
                                        kernel_size=3, stride=1, padding=1, bias=False)
                block.bn2 = nn.BatchNorm1d(block.bn2.num_features)
                if block.downsample is not None:
                    conv = nn.Conv1d(block.downsample[0].in_channels,
                                     block.downsample[0].out_channels,
                                     kernel_size=1, stride=block.downsample[0].stride[0], bias=False)
                    bn = nn.BatchNorm1d(block.downsample[1].num_features)
                    block.downsample = nn.Sequential(conv, bn)

    def forward(self, x):  # x: (B, T, D)
        x = x.permute(0, 2, 1)  # → (B, D, T)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.se_blocks[0](x)

        x = self.layer2(x)
        x = self.se_blocks[1](x)

        x = self.layer3(x)
        x = self.se_blocks[2](x)

        x = self.layer4(x)
        x = self.se_blocks[3](x)

        x = self.global_pool(x).squeeze(-1)
        x = self.classifier(x)
        return x

### ResNet 18 - 1D SE (Standard Ver.)

In [11]:
class SEBlock1D(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock1D, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class SEBasicBlock1D(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, reduction=16):
        super(SEBasicBlock1D, self).__init__()
        self.conv1 = nn.Conv1d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.se = SEBlock1D(planes, reduction)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.se(out)  # 應用SE模塊在殘差連接之前

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual  # 殘差連接
        out = self.relu(out)

        return out

class ResNet18_1D_SE_Standard(nn.Module):
    def __init__(self, input_channels: int, output_size: int):
        super(ResNet18_1D_SE_Standard, self).__init__()
        
        self.inplanes = 64
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        
        # 使用自定義的 SEBasicBlock1D 來構建網絡層
        self.layer1 = self._make_layer(SEBasicBlock1D, 64, 2)
        self.layer2 = self._make_layer(SEBasicBlock1D, 128, 2, stride=2)
        self.layer3 = self._make_layer(SEBasicBlock1D, 256, 2, stride=2)
        self.layer4 = self._make_layer(SEBasicBlock1D, 512, 2, stride=2)
        
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(512, output_size)
    
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    def forward(self, x):  # x: (B, T, D)
        x = x.permute(0, 2, 1)  # → (B, D, T)
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.global_pool(x).squeeze(-1)  # → (B, 512)
        x = self.classifier(x)               # → (B, num_classes)
        
        return x

### ResNet18 - 1D HighRes

In [12]:
class ResNet18_1D_HighRes(nn.Module):
    def __init__(self, input_channels: int, output_size: int):
        super(ResNet18_1D_HighRes, self).__init__()
        base_model = resnet18(pretrained=False)

        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=7, stride=1, padding=3, bias=False)  # stride 改 1
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.Identity()  # 移除 maxpool 降採樣

        self.layer1 = base_model.layer1
        self.layer2 = base_model.layer2
        self.layer3 = base_model.layer3
        self.layer4 = base_model.layer4

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Linear(512, output_size)

        self._convert_layers_to_1d()

    def _convert_layers_to_1d(self):
        for name in ['layer1', 'layer2', 'layer3', 'layer4']:
            layer = getattr(self, name)
            for block in layer:
                block.conv1 = nn.Conv1d(block.conv1.in_channels, block.conv1.out_channels,
                                        kernel_size=3, stride=block.conv1.stride[0],
                                        padding=1, bias=False)
                block.bn1 = nn.BatchNorm1d(block.bn1.num_features)
                block.conv2 = nn.Conv1d(block.conv2.in_channels, block.conv2.out_channels,
                                        kernel_size=3, stride=1, padding=1, bias=False)
                block.bn2 = nn.BatchNorm1d(block.bn2.num_features)
                if block.downsample is not None:
                    conv = nn.Conv1d(block.downsample[0].in_channels,
                                     block.downsample[0].out_channels,
                                     kernel_size=1, stride=block.downsample[0].stride[0], bias=False)
                    bn = nn.BatchNorm1d(block.downsample[1].num_features)
                    block.downsample = nn.Sequential(conv, bn)

    def forward(self, x):  # x: (B, T, D)
        x = x.permute(0, 2, 1)  # → (B, D, T)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)  # 此處為 Identity，等於不作用

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.global_pool(x).squeeze(-1)
        x = self.classifier(x)
        return x


## __Training and validation function__

### Extra Function

In [13]:
def compute_classwise_accuracy(student_logits_flat, y_batch, class_correct, class_total):
    """
    Computes per-class accuracy by accumulating correct and total samples for each class using vectorized operations.
    
    Args:
        student_logits_flat (torch.Tensor): Model predictions (logits) in shape [batch_size * seq_len, output_size]
        y_batch (torch.Tensor): True labels in shape [batch_size * seq_len]
        class_correct (dict): Dictionary to store correct predictions per class
        class_total (dict): Dictionary to store total samples per class
    """
    # Ensure inputs are on the same device
    if student_logits_flat.device != y_batch.device:
        raise ValueError("student_logits_flat and y_batch must be on the same device")

    # Convert logits to predicted class indices
    predictions = torch.argmax(student_logits_flat, dim=-1)  # Shape: [batch_size * seq_len]

    # Compute correct predictions mask
    correct_mask = (predictions == y_batch)  # Shape: [batch_size * seq_len], boolean

    # Get unique labels in this batch
    unique_labels = torch.unique(y_batch)

    # Update class_total and class_correct using vectorized operations
    for label in unique_labels:
        label = label.item()  # Convert tensor to scalar
        if label not in class_total:
            class_total[label] = 0
            class_correct[label] = 0
        
        # Count total samples for this label
        label_mask = (y_batch == label)
        class_total[label] += label_mask.sum().item()
        
        # Count correct predictions for this label
        class_correct[label] += (label_mask & correct_mask).sum().item()

In [14]:
def get_model_parameter_info(model):
    total_params = sum(p.numel() for p in model.parameters())
    param_size_bytes = total_params * 4
    param_size_MB = param_size_bytes / (1024**2)
    return total_params, param_size_MB

In [34]:
# 從第一個附件保留的數據增強函數
def augment_ecg(signal, sigma=0.05, shift_max=20):
    """
    對ECG信號進行數據增強
    
    Args:
        signal: 形狀為 (B, T, C) 的ECG信號
        sigma: 噪聲標準差
        shift_max: 最大時間偏移量
    
    Returns:
        增強後的信號
    """
    # 添加噪聲
    noise = np.random.normal(0, sigma, signal.shape)
    signal_noisy = signal + noise
    
    # 隨機時間偏移
    shift = np.random.randint(-shift_max, shift_max)
    if shift > 0:
        signal_shifted = np.pad(signal_noisy[:, :-shift, :], ((0, 0), (shift, 0), (0, 0)), mode='edge')
    elif shift < 0:
        signal_shifted = np.pad(signal_noisy[:, -shift:, :], ((0, 0), (0, -shift), (0, 0)), mode='edge')
    else:
        signal_shifted = signal_noisy
        
    # 縮放幅度 (±10%)
    scale = np.random.uniform(0.9, 1.1)
    signal_scaled = signal_shifted * scale
    
    return signal_scaled

# 從第一個附件保留的ECG數據集類
class ECGDataset(Dataset):
    def __init__(self, X, y, augment=False, device=None):
        """
        ECG數據集類，支持數據增強
        
        Args:
            X: 輸入數據，形狀為 (N, T, C)
            y: 標籤
            augment: 是否使用數據增強
            device: 設備(CPU/GPU)
        """
        self.X = X
        self.y = y
        self.augment = augment
        self.device = device
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        x = self.X[idx].copy()  # 創建副本以避免修改原始數據
        
        if self.augment and np.random.rand() > 0.5:  # 50% 的概率進行增強
            x = augment_ecg(x[np.newaxis, ...])[0]  # 增加和移除 batch 維度
            
        x_tensor = torch.FloatTensor(x)
        y_tensor = torch.LongTensor([self.y[idx]])[0]
        
        if self.device:
            x_tensor = x_tensor.to(self.device)
            y_tensor = y_tensor.to(self.device)
            
        return x_tensor, y_tensor

### Training Function

In [35]:
def train_model_general_classifier(model, output_size, criterion, optimizer,
                                   X_train, y_train, X_val, y_val, scheduler=None,
                                   num_epochs=10, batch_size=64, model_saving_folder=None,
                                   model_name=None, stop_signal_file=None, device=None,
                                   data_argumentation=False):

    print("\n🚀 'train_model_general_classifier' started.")
    start_time = time.time()

    # === Folder Setup ===
    if model_saving_folder:
        if os.path.exists(model_saving_folder):
            shutil.rmtree(model_saving_folder)
            print(f"✅ Removed existing folder: {model_saving_folder}")
        os.makedirs(model_saving_folder, exist_ok=True)

    model_name = model_name or 'model'
    model_saving_folder = model_saving_folder or './saved_models'
    device = device

    # === Tensor Conversion ===
    if data_argumentation:
        print("🔄 Data Augmentation Enabled")
        train_dataset = ECGDataset(X_train, y_train, augment=True, device=device)
        val_dataset   = ECGDataset(X_val, y_val, augment=False, device=device)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    else:
        print("🔄 Data Augmentation Disabled")
        X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
        y_train = torch.tensor(y_train, dtype=torch.long).to(device)
        X_val = torch.tensor(X_val, dtype=torch.float32).to(device)
        y_val = torch.tensor(y_val, dtype=torch.long).to(device)

        train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, shuffle=False)

    print("\n✅ Data Overview:")
    print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")

    best_results = []
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        class_correct, class_total = {}, {}

        if stop_signal_file and os.path.exists(stop_signal_file):
            print("\n🛑 Stop signal detected. Exiting training loop.")
            break

        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)  # (B, C)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * X_batch.size(0)
            compute_classwise_accuracy(outputs, y_batch, class_correct, class_total)

        train_loss = epoch_loss / len(train_loader.dataset)
        train_acc = {int(c): f"{(class_correct[c] / class_total[c]) * 100:.2f}%" if class_total[c] > 0 else "0.00%"
                     for c in sorted(class_total.keys())}

        # === Validation ===
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        val_class_correct, val_class_total = {}, {}
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                outputs = model(X_batch)
                val_loss += criterion(outputs, y_batch).item() * X_batch.size(0)
                predictions = torch.argmax(outputs, dim=-1)
                val_correct += (predictions == y_batch).sum().item()
                val_total += y_batch.size(0)
                compute_classwise_accuracy(outputs, y_batch, val_class_correct, val_class_total)

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / val_total
        val_acc_cls = {int(c): f"{(val_class_correct[c] / val_class_total[c]) * 100:.2f}%" if val_class_total[c] > 0 else "0.00%"
                       for c in sorted(val_class_total.keys())}

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.6f}, Train-Class-Acc: {train_acc}")
        print(f"Val Loss: {val_loss:.6f}, Val Acc: {val_acc * 100:.2f}%, Val-Class-Acc: {val_acc_cls}, LR: {optimizer.param_groups[0]['lr']:.6f}")

        model_path = os.path.join(model_saving_folder, f"{model_name}_epoch_{epoch+1}.pth")
        current = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'train_classwise_accuracy': train_acc,
            'val_classwise_accuracy': val_acc_cls,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'learning_rate': optimizer.param_groups[0]['lr'],
            'model_path': model_path
        }

        if len(best_results) < 5 or val_acc > best_results[-1]['val_accuracy']:
            if len(best_results) == 5:
                to_remove = best_results.pop()
                if os.path.exists(to_remove['model_path']):
                    os.remove(to_remove['model_path'])
                    print(f"🗑 Removed: {to_remove['model_path']}")
            best_results.append(current)
            best_results.sort(key=lambda x: (x['val_accuracy'], x['epoch']), reverse=True)
            torch.save(current, model_path)
            print(f"✅ Saved model: {model_path}")

        if scheduler:
            scheduler.step(val_loss)

    end_time = time.time()
    training_time = end_time - start_time
    total_params, param_size_MB = get_model_parameter_info(model)

    if best_results:
        best = best_results[0]
        best_model_path = os.path.join(model_saving_folder, f"{model_name}_best.pth")
        torch.save(best, best_model_path)
        print(f"\n🏆 Best model saved as: {best_model_path} (Val Accuracy: {best['val_accuracy'] * 100:.2f}%)")

    final_model_path = os.path.join(model_saving_folder, f"{model_name}_final.pth")
    torch.save(current, final_model_path)
    print(f"\n📌 Final model saved as: {final_model_path}")

    print("\n🎯 Top 5 Best Models:")
    for res in best_results:
        print(f"Epoch {res['epoch']}, Train Loss: {res['train_loss']:.6f}, Train-Acc: {res['train_classwise_accuracy']},\n"
              f"Val Loss: {res['val_loss']:.6f}, Val Acc: {res['val_accuracy']*100:.2f}%, Val-Class-Acc: {res['val_classwise_accuracy']},"
              f" Model Path: {res['model_path']}")

    print(f"\n🧠 Model Summary:")
    print(f"Total Parameters: {total_params:,}")
    print(f"Model Size (float32): {param_size_MB:.2f} MB")
    print(f"Total Training Time: {training_time:.2f} seconds")

    # 🔥 Cleanup
    del X_train, y_train, X_val, y_val, train_loader, val_loader, current, outputs, predictions
    del loss, val_loss, val_correct, val_total, class_correct, class_total, val_class_correct, val_class_total
    del model, optimizer, criterion, scheduler
    torch.cuda.empty_cache()
    gc.collect()

    return {
        'training_time_sec': training_time,
        'total_params': total_params,
        'model_size_MB': param_size_MB,
        'best_val_accuracy': best_results[0]['val_accuracy'] if best_results else None,
        'val_classwise_accuracy': best_results[0]['val_classwise_accuracy'] if best_results else None,
        'best_model_path': best_model_path if best_results else None,
        'final_model_path': final_model_path
    }


## __Test Model__

## ResNet18_1D

### No Data augment

In [19]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_v1"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_1D',
    stop_signal_file=stop_signal_file,
    device=device
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 25197 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]


/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_v1
🔄 Data Augmentation Disabled

✅ Data Overview:
X_train: torch.Size([1468, 5000, 12]), y_train: torch.Size([1468])
X_val: torch.Size([368, 5000, 12]), y_val: torch.Size([368])
Epoch 1/200, Train Loss: 0.471980, Train-Class-Acc: {0: '81.20%', 1: '72.62%'}
Val Loss: 0.545346, Val Acc: 76.63%, Val-Class-Acc: {0: '85.87%', 1: '67.39%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_v1/ResNet18_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.386238, Train-Class-Acc: {0: '85.29%', 1: '78.88%'}
Val Loss: 0.946580, Val Acc: 67.12%, Val-Class-Acc: {0: '35.87%', 1: '98.37%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_v1/ResNet18_1D_epoch_2.pth
Epoch 3/200, Train Loss: 0.352740, Train-Class-Acc: {0: '86.78%', 1: '80.52%'}
Val Loss: 0.392736, Val Acc: 84.24%, Val-Class-Acc: {0: '84.78%', 1: 

0

### Data augment

In [20]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_v2"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_1D',
    stop_signal_file=stop_signal_file,
    device=device,
    data_argumentation=True
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 32751 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]


/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_v2
🔄 Data Augmentation Enabled

✅ Data Overview:
X_train: (1468, 5000, 12), y_train: (1468,)
X_val: (368, 5000, 12), y_val: (368,)
Epoch 1/200, Train Loss: 0.489694, Train-Class-Acc: {0: '78.34%', 1: '74.25%'}
Val Loss: 0.479197, Val Acc: 79.62%, Val-Class-Acc: {0: '90.22%', 1: '69.02%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_v2/ResNet18_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.377966, Train-Class-Acc: {0: '87.47%', 1: '77.38%'}
Val Loss: 0.494350, Val Acc: 78.26%, Val-Class-Acc: {0: '64.67%', 1: '91.85%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_v2/ResNet18_1D_epoch_2.pth
Epoch 3/200, Train Loss: 0.353156, Train-Class-Acc: {0: '87.74%', 1: '79.43%'}
Val Loss: 0.383514, Val Acc: 84.24%, Val-Class-Acc: {0: '91.85%', 1: '76.63%'}, LR: 0.001000
✅ Saved model: Class_In

0

## ResNet18_1D_big_ker

### No Data augment (ResNet18_1D_big_ker)

In [39]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_big_ker_v1"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D_big_ker(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_big_ker_1D',
    stop_signal_file=stop_signal_file,
    device=device
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 663 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]


/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_ker_v1
🔄 Data Augmentation Disabled

✅ Data Overview:
X_train: torch.Size([1468, 5000, 12]), y_train: torch.Size([1468])
X_val: torch.Size([368, 5000, 12]), y_val: torch.Size([368])
Epoch 1/200, Train Loss: 0.495284, Train-Class-Acc: {0: '75.89%', 1: '73.57%'}
Val Loss: 0.541140, Val Acc: 67.12%, Val-Class-Acc: {0: '42.93%', 1: '91.30%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_ker_v1/ResNet18_big_ker_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.390758, Train-Class-Acc: {0: '86.24%', 1: '77.38%'}
Val Loss: 0.681212, Val Acc: 75.54%, Val-Class-Acc: {0: '96.20%', 1: '54.89%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_ker_v1/ResNet18_big_ker_1D_epoch_2.pth
Epoch 3/200, Train Loss: 0.369286, Train-Class-Acc: {0: '85.42%', 1: '78.47%'}
Val Loss: 0.430852, Val Acc: 

0

### Data augment (ResNet18_1D_big_ker)

In [44]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_big_ker_v2"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D_big_ker(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_big_ker_1D',
    stop_signal_file=stop_signal_file,
    device=device,
    data_argumentation=True
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 432 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]

🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_ker_v2
🔄 Data Augmentation Enabled

✅ Data Overview:
X_train: (1468, 5000, 12), y_train: (1468,)
X_val: (368, 5000, 12), y_val: (368,)


/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/200, Train Loss: 0.512468, Train-Class-Acc: {0: '78.88%', 1: '70.44%'}
Val Loss: 0.892764, Val Acc: 67.39%, Val-Class-Acc: {0: '41.30%', 1: '93.48%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_ker_v2/ResNet18_big_ker_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.389581, Train-Class-Acc: {0: '86.38%', 1: '77.52%'}
Val Loss: 0.418705, Val Acc: 82.34%, Val-Class-Acc: {0: '84.78%', 1: '79.89%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_ker_v2/ResNet18_big_ker_1D_epoch_2.pth
Epoch 3/200, Train Loss: 0.355415, Train-Class-Acc: {0: '86.78%', 1: '81.20%'}
Val Loss: 0.466924, Val Acc: 79.08%, Val-Class-Acc: {0: '72.28%', 1: '85.87%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_ker_v2/ResNet18_big_ker_1D_epoch_3.pth
Epoch 4/200, Train Loss: 0.375461, Train-Class-Acc: {0: '86.78%', 1: '79.02%'}
Val Loss: 0.483078, Val Acc: 78.53%, Val-Class-Acc: {0: '

0

## ResNet18_1D_big_inplane

### No Data augment (ResNet18_1D_big_inplane)

In [42]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_big_inplane_v1"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D_big_inplane(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_big_inplane_1D',
    stop_signal_file=stop_signal_file,
    device=device
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 432 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]

🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_inplane_v1
🔄 Data Augmentation Disabled

✅ Data Overview:
X_train: torch.Size([1468, 5000, 12]), y_train: torch.Size([1468])
X_val: torch.Size([368, 5000, 12]), y_val: torch.Size([368])
Epoch 1/200, Train Loss: 1.337290, Train-Class-Acc: {0: '73.30%', 1: '67.71%'}
Val Loss: 0.694106, Val Acc: 70.11%, Val-Class-Acc: {0: '96.20%', 1: '44.02%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_inplane_v1/ResNet18_big_inplane_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.454535, Train-Class-Acc: {0: '84.88%', 1: '73.84%'}
Val Loss: 0.526020, Val Acc: 80.43%, Val-Class-Acc: {0: '77.17%', 1: '83.70%'}, LR: 0.001000
✅ Save

0

### Data augment (ResNet18_1D_big_inplane)

In [43]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_big_inplane_v2"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D_big_inplane(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_big_inplane_1D',
    stop_signal_file=stop_signal_file,
    device=device,
    data_argumentation=True
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 409 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]

🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_inplane_v2
🔄 Data Augmentation Enabled

✅ Data Overview:
X_train: (1468, 5000, 12), y_train: (1468,)
X_val: (368, 5000, 12), y_val: (368,)
Epoch 1/200, Train Loss: 1.497168, Train-Class-Acc: {0: '72.48%', 1: '68.94%'}
Val Loss: 0.472006, Val Acc: 78.26%, Val-Class-Acc: {0: '84.78%', 1: '71.74%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_big_inplane_v2/ResNet18_big_inplane_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.476889, Train-Class-Acc: {0: '83.11%', 1: '75.89%'}
Val Loss: 0.476844, Val Acc: 80.98%, Val-Class-Acc: {0: '88.59%', 1: '73.37%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18

0

## ResNet18_1D_SE

### No Data augment (SE)

In [24]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_SE_v1"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D_SE(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_SE_1D',
    stop_signal_file=stop_signal_file,
    device=device
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 25574 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]


/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_SE_v1
🔄 Data Augmentation Disabled

✅ Data Overview:
X_train: torch.Size([1468, 5000, 12]), y_train: torch.Size([1468])
X_val: torch.Size([368, 5000, 12]), y_val: torch.Size([368])
Epoch 1/200, Train Loss: 0.485963, Train-Class-Acc: {0: '81.06%', 1: '68.80%'}
Val Loss: 0.529063, Val Acc: 77.99%, Val-Class-Acc: {0: '95.65%', 1: '60.33%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_SE_v1/ResNet18_SE_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.371485, Train-Class-Acc: {0: '85.69%', 1: '79.70%'}
Val Loss: 0.637430, Val Acc: 73.91%, Val-Class-Acc: {0: '57.61%', 1: '90.22%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_SE_v1/ResNet18_SE_1D_epoch_2.pth
Epoch 3/200, Train Loss: 0.365066, Train-Class-Acc: {0: '86.78%', 1: '80.79%'}
Val Loss: 0.435498, Val Acc: 81.52%, Val-Class-Acc: {0

0

### Data augment (SE)

In [17]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_SE_v2"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D_SE(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_SE_1D',
    stop_signal_file=stop_signal_file,
    device=device,
    data_argumentation=True
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 1
    - Memory Used    : 14709 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]


/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_SE_v2
🔄 Data Augmentation Enabled

✅ Data Overview:
X_train: (1468, 5000, 12), y_train: (1468,)
X_val: (368, 5000, 12), y_val: (368,)
Epoch 1/200, Train Loss: 0.494351, Train-Class-Acc: {0: '80.79%', 1: '70.71%'}
Val Loss: 0.621876, Val Acc: 75.54%, Val-Class-Acc: {0: '96.74%', 1: '54.35%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_SE_v2/ResNet18_SE_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.380200, Train-Class-Acc: {0: '86.10%', 1: '80.79%'}
Val Loss: 0.357954, Val Acc: 83.42%, Val-Class-Acc: {0: '83.15%', 1: '83.70%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_SE_v2/ResNet18_SE_1D_epoch_2.pth
Epoch 3/200, Train Loss: 0.353001, Train-Class-Acc: {0: '87.47%', 1: '80.65%'}
Val Loss: 0.394147, Val Acc: 83.15%, Val-Class-Acc: {0: '78.80%', 1: '87.50%'}, LR: 0.001000
✅ Saved 

0

## ResNet18_1D_SE_Standard

### No Data augment (SE std)

In [18]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_SE_std_v1"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D_SE_Standard(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_SE_std_1D',
    stop_signal_file=stop_signal_file,
    device=device
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 16904 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]

🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_SE_std_v1
🔄 Data Augmentation Disabled

✅ Data Overview:
X_train: torch.Size([1468, 5000, 12]), y_train: torch.Size([1468])
X_val: torch.Size([368, 5000, 12]), y_val: torch.Size([368])
Epoch 1/200, Train Loss: 0.468331, Train-Class-Acc: {0: '82.83%', 1: '70.84%'}
Val Loss: 0.448086, Val Acc: 80.16%, Val-Class-Acc: {0: '69.57%', 1: '90.76%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_SE_std_v1/ResNet18_SE_std_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.369743, Train-Class-Acc: {0: '86.78%', 1: '79.16%'}
Val Loss: 0.872699, Val Acc: 65.76%, Val-Class-Acc: {0: '33.70%', 1: '97.83%'}, LR: 0.001000
✅ Saved model: Clas

0

### Data augment (SE std)

In [19]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_SE_std_v2"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D_SE_Standard(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_SE_std_1D',
    stop_signal_file=stop_signal_file,
    device=device,
    data_argumentation=True
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 17541 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]

🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_SE_std_v2
🔄 Data Augmentation Enabled

✅ Data Overview:
X_train: (1468, 5000, 12), y_train: (1468,)
X_val: (368, 5000, 12), y_val: (368,)
Epoch 1/200, Train Loss: 0.473647, Train-Class-Acc: {0: '80.38%', 1: '72.34%'}
Val Loss: 0.436457, Val Acc: 79.08%, Val-Class-Acc: {0: '77.17%', 1: '80.98%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_SE_std_v2/ResNet18_SE_std_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.388787, Train-Class-Acc: {0: '86.92%', 1: '79.56%'}
Val Loss: 0.466764, Val Acc: 82.88%, Val-Class-Acc: {0: '84.24%', 1: '81.52%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/Re

0

## ResNet18_1D_HighRes

### No Data augment (HighRes)

In [20]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_HighRes_v1"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D_HighRes(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_HighRes_1D',
    stop_signal_file=stop_signal_file,
    device=device
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 2
    - Memory Used    : 38 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]


/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_HighRes_v1
🔄 Data Augmentation Disabled

✅ Data Overview:
X_train: torch.Size([1468, 5000, 12]), y_train: torch.Size([1468])
X_val: torch.Size([368, 5000, 12]), y_val: torch.Size([368])
Epoch 1/200, Train Loss: 0.484376, Train-Class-Acc: {0: '80.79%', 1: '70.71%'}
Val Loss: 0.975452, Val Acc: 68.75%, Val-Class-Acc: {0: '40.22%', 1: '97.28%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_HighRes_v1/ResNet18_HighRes_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.401870, Train-Class-Acc: {0: '85.56%', 1: '73.71%'}
Val Loss: 0.434427, Val Acc: 79.62%, Val-Class-Acc: {0: '91.85%', 1: '67.39%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_HighRes_v1/ResNet18_HighRes_1D_epoch_2.pth
Epoch 3/200, Train Loss: 0.366478, Train-Class-Acc: {0: '86.92%', 1: '80.38%'}
Val Loss: 0.417264, Val Acc: 

0

### Data augment (HighRes)

In [36]:
# ==== Load Period 1 Data ====
X_train = np.load(os.path.join(save_dir, "X_train_p1.npy"))  # Shape: (B, 5000, 12)
y_train = np.load(os.path.join(save_dir, "y_train_p1.npy"))
X_test = np.load(os.path.join(save_dir, "X_test_p1.npy"))
y_test = np.load(os.path.join(save_dir, "y_test_p1.npy"))

# ==== Model Hyperparameters ====
input_channels = X_train.shape[2]                  # 12 leads
output_size = len(np.unique(y_train))              # Number of classes (e.g., 2 for Period 1)
num_epochs = 200
batch_size = 64                                  # Not needed for ResNet18_1D
device = auto_select_cuda_device()

print("✅ input shape:", X_train.shape)
print("✅ unique y_train:", np.unique(y_train))
print("✅ unique y_test :", np.unique(y_test))
assert np.max(y_train) < output_size
assert np.max(y_test) < output_size

# ==== Paths ====
stop_signal_file = os.path.normpath(os.path.join(
    'Class_Incremental_CL', 'CPSC_CIL/stop_training.txt'
))
model_saving_folder = os.path.normpath(os.path.join(
    'Class_Incremental_CL', "CPSC_CIL/ResNet18_Selection/ResNet18_HighRes_v2"
))
ensure_folder(model_saving_folder)

# ==== Model ====
model = ResNet18_1D_HighRes(input_channels=input_channels, output_size=output_size).to(device)

# ==== Optimizer and Training ====
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.9, patience=10)

# ==== Train ====
result_summary = train_model_general_classifier(
    model=model,
    output_size=output_size,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train,
    y_train=y_train,
    X_val=X_test,
    y_val=y_test,
    scheduler=scheduler,
    num_epochs=num_epochs,
    batch_size=batch_size,
    model_saving_folder=model_saving_folder,
    model_name='ResNet18_HighRes_1D',
    stop_signal_file=stop_signal_file,
    device=device,
    data_argumentation=True
)

# ==== Cleanup ====
del model, X_train, y_train, X_test, y_test
torch.cuda.empty_cache()
gc.collect()

🎯 Automatically selected GPU:
    - CUDA Device ID : 0
    - Memory Used    : 626 MiB
    - Device Name    : NVIDIA RTX A6000
✅ input shape: (1468, 5000, 12)
✅ unique y_train: [0 1]
✅ unique y_test : [0 1]


/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asus/anaconda3/envs/CIL_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)



🚀 'train_model_general_classifier' started.
✅ Removed existing folder: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_HighRes_v2
🔄 Data Augmentation Enabled

✅ Data Overview:
X_train: (1468, 5000, 12), y_train: (1468,)
X_val: (368, 5000, 12), y_val: (368,)
Epoch 1/200, Train Loss: 0.483207, Train-Class-Acc: {0: '79.70%', 1: '72.07%'}
Val Loss: 0.623583, Val Acc: 75.00%, Val-Class-Acc: {0: '93.48%', 1: '56.52%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_HighRes_v2/ResNet18_HighRes_1D_epoch_1.pth
Epoch 2/200, Train Loss: 0.392023, Train-Class-Acc: {0: '83.79%', 1: '77.93%'}
Val Loss: 0.488012, Val Acc: 75.27%, Val-Class-Acc: {0: '67.93%', 1: '82.61%'}, LR: 0.001000
✅ Saved model: Class_Incremental_CL/CPSC_CIL/ResNet18_Selection/ResNet18_HighRes_v2/ResNet18_HighRes_1D_epoch_2.pth
Epoch 3/200, Train Loss: 0.375504, Train-Class-Acc: {0: '88.15%', 1: '75.89%'}
Val Loss: 0.418004, Val Acc: 80.71%, Val-Class-Acc: {0: '85.87%', 1: '75.54%

0

## 🧪 CPSC - ResNet18 Architecture Variants

| Model Variant                  | Total Params | Model Size | Training Time (s) | Val Acc | Class-wise Accuracy                 |
|-------------------------------|--------------|------------|--------------------|---------|-------------------------------------|
| ResNet18_1D                   | 3,849,858    | 14.69 MB   | 292.84             | 87.77%  | {0: 90.22%, 1: 85.33%}              |
| ResNet18_1D_big_ker           | 3,856,002    | 14.71 MB   | 133.14             | 87.77%  | {0: 90.76%, 1: 84.78%}              |
| **ResNet18_1D_big_inplane**   | **3,857,026**| **14.71 MB** | **134.66**       | **88.86%** | **{0: 91.85%, 1: 85.87%}**          |
| ResNet18_SE                   | 3,893,378    | 14.85 MB   | 228.37             | 86.96%  | {0: 91.30%, 1: 82.61%}              |
| ResNet18_1D_SE_Standard       | 3,936,898    | 15.02 MB   | 170.17             | 86.14%  | {0: 90.22%, 1: 82.07%}              |
| ResNet18_1D_HighRes           | 3,849,858    | 14.69 MB   | 437.03             | 83.97%  | {0: 82.61%, 1: 85.33%}              |
